In [3]:
pip install pandas numpy matplotlib seaborn jupyter scikit-learn pdfplumber geopandas contextily fiona


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge



In [5]:
cost = pd.read_csv('interconnectionNY_cost.csv')

In [6]:
cost.columns
cost = cost.drop(columns=['Queue ID 2'])

In [7]:
cost

,Queue ID,Queue Date,State,County,Resource Type,Nameplate MW,Interconnecting Transmission Owner,Study Type,Study Date,Request Status,$2022 POI Cost/kW,$2022 Network Cost/kW,$2022 Total Cost/kW
0,142,10/30/03,NY,Steuben,Wind Land,50.60,New York State Electric and Gas,System Impact,5/31/06,Withdrawn,$104.37,$-,$104.37
1,144,2/18/04,NY,Wyoming,Wind Land,129.00,New York State Electric and Gas,Facilities - Part 2 (Class Year),9/2/08,Complete,$2.82,$77.91,$80.73
2,147,4/16/04,NY,Madison,Wind Land,41.40,National Grid,Feasibility,6/12/06,Withdrawn,$92.76,$-,$92.76
3,150,6/17/04,NY,Ostego,Wind Land,70.00,National Grid,System Impact,11/24/08,Withdrawn,$43.90,$13.72,$57.62
4,152,7/23/04,NY,Delaware,Wind Land,129.00,New York State Electric and Gas,System Impact,5/12/06,Withdrawn,$40.21,$-,$40.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,965,12/21/19,NY,Suffolk,Storage,76.86,Long Island Power Authority,System Impact,3/12/21,Active,$7.49,$157.15,$164.64
290,967,12/27/19,NY,Ulster,Storage,94.00,Central Hudson Gas & Electric,System Impact,1/14/21,Active,$29.56,$-,$29.56
291,971,1/10/20,NY,Suffolk,Storage,293.50,Long Island Power Authority,System Impact,3/4/21,Active,$19.37,$24.90,$44.28
292,994,2/12/20,NY,Suffolk,Storage,90.00,Long Island Power Authority,System Impact,12/18/20,Active,$66.64,$85.66,$152.30


In [8]:
for col in ["$2022 POI Cost/kW", "$2022 Network Cost/kW", "$2022 Total Cost/kW"]:
    cost[col] = (
        cost[col]
        .astype(str)
        .str.replace("$", "", regex=False)
        .str.replace(",", "", regex=False)
        .str.replace(r"^\s*-\s*$", "0.00", regex=True)
        .astype(float)
    )
cost = cost.dropna(subset=["$2022 POI Cost/kW"])

for i in ["Study Date", "Queue Date"]:
    cost[i] = pd.to_datetime(cost[i], errors="coerce")
    cost[i[0] + "_" + "Year"] = cost[i].dt.year
    cost[i[0] + "_" + "Month"] = cost[i].dt.month
    cost[i[0] + "_" + "Day"] = cost[i].dt.day
     #separated each date value to own column

cost_cleaned = cost.drop(columns=["State", "Study Date", "Queue Date"])



/var/folders/z4/3qsrn4ms31dd0qvvyxllm0jm0000gn/T/ipykernel_84275/4160514672.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cost[i] = pd.to_datetime(cost[i], errors="coerce")
/var/folders/z4/3qsrn4ms31dd0qvvyxllm0jm0000gn/T/ipykernel_84275/4160514672.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  cost[i] = pd.to_datetime(cost[i], errors="coerce")


In [9]:
cost_cleaned

,Queue ID,County,Resource Type,Nameplate MW,Interconnecting Transmission Owner,Study Type,Request Status,$2022 POI Cost/kW,$2022 Network Cost/kW,$2022 Total Cost/kW,S_Year,S_Month,S_Day,Q_Year,Q_Month,Q_Day
0,142,Steuben,Wind Land,50.60,New York State Electric and Gas,System Impact,Withdrawn,104.37,0.00,104.37,2006,5,31,2003,10,30
1,144,Wyoming,Wind Land,129.00,New York State Electric and Gas,Facilities - Part 2 (Class Year),Complete,2.82,77.91,80.73,2008,9,2,2004,2,18
2,147,Madison,Wind Land,41.40,National Grid,Feasibility,Withdrawn,92.76,0.00,92.76,2006,6,12,2004,4,16
3,150,Ostego,Wind Land,70.00,National Grid,System Impact,Withdrawn,43.90,13.72,57.62,2008,11,24,2004,6,17
4,152,Delaware,Wind Land,129.00,New York State Electric and Gas,System Impact,Withdrawn,40.21,0.00,40.21,2006,5,12,2004,7,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,965,Suffolk,Storage,76.86,Long Island Power Authority,System Impact,Active,7.49,157.15,164.64,2021,3,12,2019,12,21
290,967,Ulster,Storage,94.00,Central Hudson Gas & Electric,System Impact,Active,29.56,0.00,29.56,2021,1,14,2019,12,27
291,971,Suffolk,Storage,293.50,Long Island Power Authority,System Impact,Active,19.37,24.90,44.28,2021,3,4,2020,1,10
292,994,Suffolk,Storage,90.00,Long Island Power Authority,System Impact,Active,66.64,85.66,152.30,2020,12,18,2020,2,12


In [10]:
cost_cleaned.columns

Index(['Queue ID', 'County', 'Resource Type', 'Nameplate MW',
       'Interconnecting Transmission Owner', 'Study Type', 'Request Status',
       '$2022 POI Cost/kW', '$2022 Network Cost/kW', '$2022 Total Cost/kW',
       'S_Year', 'S_Month', 'S_Day', 'Q_Year', 'Q_Month', 'Q_Day'],
      dtype='object')

In [11]:
target = "$2022 POI Cost/kW"


X = cost.drop(columns=[target])
y = cost[target]

#Preprocessing
num_features = ["Queue ID", "Nameplate MW", '$2022 Network Cost/kW', 'S_Year', 'S_Month', 'S_Day', 'Q_Year', 'Q_Month', 'Q_Day']
cat_features = ['County', 'Resource Type', 'Nameplate MW', 'Interconnecting Transmission Owner', 'Study Type', 'Request Status']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=67
)

In [ ]:
models = {
    "RandomForest": {
        "model": RandomForestRegressor(),
        "params": {
            "n_estimators": [100, 200, 300, 400, 500],
            "max_depth": [5, 10, None, 15, 3],
            "min_samples_split": [2, 5, 6, 9],
            "min_samples_leaf": [1, 2, 7, 4],
        }
    },
    "RidgeRegression": {
        "model": Ridge(),
        "params": {
            "alpha": [0.01, 0.1, 1, 10, 20], 
            "solver": ["auto", "svd", "cholesky", "lsqr"]
        }
    },
    "GradientBoosting": {
        "model": GradientBoostingRegressor(),
        "params": {
            "n_estimators": [100, 200, 300, 400, 500],
            "learning_rate": [0.01, 0.05, 0.07, 0.08, 0.1],
            "max_depth": [3, 4, 5, 10, 7],
            "subsample": [0.8, 1.0, 0.5, 0.9]
        }
    }
}

#OHE

cat_pipeline = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])
preprocess = ColumnTransformer([
    ("cat", cat_pipeline, cat_features),
    ("num", "passthrough", num_features)
])

#models

best_models = {}

for name, m in models.items():
    print(f"Running GridSearchCV for {name}...")
    
    pipeline = Pipeline([
        ("preprocess", preprocess),
        ("model", m["model"])
    ])
    
    grid = GridSearchCV(
        pipeline,
        param_grid={"model__" + k: v for k, v in m["params"].items()},
        cv=5,
        scoring="neg_mean_absolute_error",   # for the regression metric
        n_jobs=-1
    )
    
    grid.fit(X_train, y_train)
    best_models[name] = grid

    y_pred = grid.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"Best RMSE for {name}: {rmse:.4f}")
    print(f"Best R² for {name}: {r2:.4f}")
    print(f"Best parameters: {grid.best_params_}\n")


Running GridSearchCV for RandomForest...
Best RMSE for RandomForest: 73.8311
Best R² for RandomForest: 0.3778
Best parameters: {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 500}

Running GridSearchCV for RidgeRegression...


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:516: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File 

Best RMSE for RidgeRegression: 61.5577
Best R² for RidgeRegression: 0.5675
Best parameters: {'model__alpha': 100, 'model__solver': 'lsqr'}

Running GridSearchCV for GradientBoosting...
Best RMSE for GradientBoosting: 78.0026
Best R² for GradientBoosting: 0.3055
Best parameters: {'model__learning_rate': 0.05, 'model__max_depth': 4, 'model__n_estimators': 200, 'model__subsample': 1.0}



In [ ]:
best parameters: Ridge Regression: alpha 100, model solver lsqr

## Sample Code

In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

class POICostPredictor:
    """
    Machine Learning model to predict Point of Interconnection (POI) costs
    for projects in the NY interconnection queue.
    """
    
    def __init__(self):
        self.model = None
        self.encoders = {}
        self.feature_columns = []
        self.metrics = {}
        
    def load_data(self, filepath):
        """Load and preprocess the interconnection queue data."""
        print(f"Loading data from {filepath}...")
        
        # Read CSV
        df = pd.read_csv(filepath)
        
        # Display basic info
        print(f"\nDataset shape: {df.shape}")
        print(f"Columns: {df.columns.tolist()}")
        
        return df
    
    def clean_data(self, df):
        """Clean and prepare data for modeling."""
        print("\nCleaning data...")
        
        # Create a copy
        df_clean = df.copy()
        
        # Clean POI Cost column (remove $ and convert to float)
        if '$2022 POI Cost/kW' in df_clean.columns:
            df_clean['POI_Cost_kW'] = df_clean['$2022 POI Cost/kW'].str.replace('$', '').astype(float)
        else:
            raise ValueError("Column '$2022 POI Cost/kW' not found in dataset")
        
        # Filter for valid costs and Active projects
        df_clean = df_clean[
            (df_clean['POI_Cost_kW'] >= 0) & 
            (df_clean['POI_Cost_kW'].notna()) &
            (df_clean['Request Status'] == 'Active')
        ]
        
        # Extract year from Queue Date
        df_clean['Queue Date'] = pd.to_datetime(df_clean['Queue Date'], errors='coerce')
        df_clean['Queue_Year'] = df_clean['Queue Date'].dt.year
        
        # Fill missing values
        categorical_cols = ['County', 'Resource Type', 'Interconnecting Transmission Owner', 'Study Type']
        for col in categorical_cols:
            if col in df_clean.columns:
                df_clean[col] = df_clean[col].fillna('Unknown')
        
        # Fill missing numerical values
        if 'Nameplate MW' in df_clean.columns:
            df_clean['Nameplate MW'] = df_clean['Nameplate MW'].fillna(df_clean['Nameplate MW'].median())
        
        print(f"Clean dataset shape: {df_clean.shape}")
        print(f"Target variable stats:\n{df_clean['POI_Cost_kW'].describe()}")
        
        return df_clean
    
    def prepare_features(self, df):
        """Prepare features for modeling."""
        print("\nPreparing features...")
        
        # Define feature columns
        self.feature_columns = ['County', 'Resource Type', 'Nameplate MW', 
                                'Interconnecting Transmission Owner', 'Study Type', 'Queue_Year']
        
        # Create feature dataframe
        X = df[self.feature_columns].copy()
        y = df['POI_Cost_kW'].copy()
        
        # Encode categorical variables
        categorical_features = ['County', 'Resource Type', 'Interconnecting Transmission Owner', 'Study Type']
        
        for col in categorical_features:
            if col not in self.encoders:
                self.encoders[col] = LabelEncoder()
                X[col] = self.encoders[col].fit_transform(X[col])
            else:
                X[col] = self.encoders[col].transform(X[col])
        
        print(f"Feature shape: {X.shape}")
        print(f"Target shape: {y.shape}")
        
        return X, y
    
    def train_model(self, X, y, model_type='random_forest', test_size=0.2, random_state=42):
        """Train the machine learning model."""
        print(f"\nTraining {model_type} model...")
        
        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )
        
        print(f"Training set size: {X_train.shape[0]}")
        print(f"Test set size: {X_test.shape[0]}")
        
        # Initialize model
        if model_type == 'random_forest':
            self.model = RandomForestRegressor(
                n_estimators=100,
                max_depth=10,
                min_samples_split=5,
                min_samples_leaf=2,
                random_state=random_state,
                n_jobs=-1
            )
        elif model_type == 'gradient_boosting':
            self.model = GradientBoostingRegressor(
                n_estimators=100,
                max_depth=5,
                learning_rate=0.1,
                random_state=random_state
            )
        elif model_type == 'linear_regression':
            self.model = LinearRegression()
        else:
            raise ValueError(f"Unknown model type: {model_type}")
        
        # Train model
        self.model.fit(X_train, y_train)
        
        # Make predictions
        y_train_pred = self.model.predict(X_train)
        y_test_pred = self.model.predict(X_test)
        
        # Calculate metrics
        self.metrics = {
            'train': {
                'mae': mean_absolute_error(y_train, y_train_pred),
                'rmse': np.sqrt(mean_squared_error(y_train, y_train_pred)),
                'r2': r2_score(y_train, y_train_pred)
            },
            'test': {
                'mae': mean_absolute_error(y_test, y_test_pred),
                'rmse': np.sqrt(mean_squared_error(y_test, y_test_pred)),
                'r2': r2_score(y_test, y_test_pred)
            }
        }
        
        # Print metrics
        print("\n" + "="*50)
        print("MODEL PERFORMANCE METRICS")
        print("="*50)
        print("\nTraining Set:")
        print(f"  MAE:  ${self.metrics['train']['mae']:.2f}/kW")
        print(f"  RMSE: ${self.metrics['train']['rmse']:.2f}/kW")
        print(f"  R²:   {self.metrics['train']['r2']:.4f}")
        print("\nTest Set:")
        print(f"  MAE:  ${self.metrics['test']['mae']:.2f}/kW")
        print(f"  RMSE: ${self.metrics['test']['rmse']:.2f}/kW")
        print(f"  R²:   {self.metrics['test']['r2']:.4f}")
        print("="*50)
        
        return X_train, X_test, y_train, y_test, y_train_pred, y_test_pred
    
    def plot_results(self, y_test, y_test_pred, df):
        """Create visualization plots."""
        print("\nGenerating plots...")
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        
        # 1. Actual vs Predicted
        axes[0, 0].scatter(y_test, y_test_pred, alpha=0.5, edgecolors='k', linewidths=0.5)
        axes[0, 0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 
                        'r--', lw=2, label='Perfect Prediction')
        axes[0, 0].set_xlabel('Actual POI Cost ($/kW)', fontsize=11)
        axes[0, 0].set_ylabel('Predicted POI Cost ($/kW)', fontsize=11)
        axes[0, 0].set_title('Actual vs Predicted POI Costs', fontsize=13, fontweight='bold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        # 2. Residual Plot
        residuals = y_test - y_test_pred
        axes[0, 1].scatter(y_test_pred, residuals, alpha=0.5, edgecolors='k', linewidths=0.5)
        axes[0, 1].axhline(y=0, color='r', linestyle='--', lw=2)
        axes[0, 1].set_xlabel('Predicted POI Cost ($/kW)', fontsize=11)
        axes[0, 1].set_ylabel('Residuals ($/kW)', fontsize=11)
        axes[0, 1].set_title('Residual Plot', fontsize=13, fontweight='bold')
        axes[0, 1].grid(True, alpha=0.3)
        
        # 3. Average POI Cost by Resource Type
        resource_avg = df.groupby('Resource Type')['POI_Cost_kW'].mean().sort_values(ascending=False)
        axes[1, 0].barh(range(len(resource_avg)), resource_avg.values, color='steelblue')
        axes[1, 0].set_yticks(range(len(resource_avg)))
        axes[1, 0].set_yticklabels(resource_avg.index)
        axes[1, 0].set_xlabel('Average POI Cost ($/kW)', fontsize=11)
        axes[1, 0].set_title('Average POI Cost by Resource Type', fontsize=13, fontweight='bold')
        axes[1, 0].grid(True, alpha=0.3, axis='x')
        
        # 4. Feature Importance (if available)
        if hasattr(self.model, 'feature_importances_'):
            importances = self.model.feature_importances_
            indices = np.argsort(importances)[::-1]
            feature_names = self.feature_columns
            
            axes[1, 1].barh(range(len(importances)), importances[indices], color='coral')
            axes[1, 1].set_yticks(range(len(importances)))
            axes[1, 1].set_yticklabels([feature_names[i] for i in indices])
            axes[1, 1].set_xlabel('Feature Importance', fontsize=11)
            axes[1, 1].set_title('Feature Importance', fontsize=13, fontweight='bold')
            axes[1, 1].grid(True, alpha=0.3, axis='x')
        else:
            axes[1, 1].text(0.5, 0.5, 'Feature importance\nnot available\nfor this model type', 
                           ha='center', va='center', fontsize=12)
            axes[1, 1].set_title('Feature Importance', fontsize=13, fontweight='bold')
        
        plt.tight_layout()
        plt.savefig('poi_cost_predictions.png', dpi=300, bbox_inches='tight')
        print("Plot saved as 'poi_cost_predictions.png'")
        plt.show()
    
    def predict_single(self, county, resource_type, nameplate_mw, transmission_owner, 
                      study_type, queue_year):
        """Make a prediction for a single project."""
        if self.model is None:
            raise ValueError("Model has not been trained yet. Call train_model() first.")
        
        # Create input dataframe
        input_data = pd.DataFrame({
            'County': [county],
            'Resource Type': [resource_type],
            'Nameplate MW': [nameplate_mw],
            'Interconnecting Transmission Owner': [transmission_owner],
            'Study Type': [study_type],
            'Queue_Year': [queue_year]
        })
        
        # Encode categorical variables
        for col in ['County', 'Resource Type', 'Interconnecting Transmission Owner', 'Study Type']:
            if col in self.encoders:
                # Handle unknown categories
                try:
                    input_data[col] = self.encoders[col].transform(input_data[col])
                except ValueError:
                    # If category not seen during training, use most common category
                    input_data[col] = 0
        
        # Make prediction
        prediction = self.model.predict(input_data)[0]
        
        return prediction
    
    def save_model(self, filepath='poi_cost_model.pkl'):
        """Save the trained model."""
        import pickle
        
        model_data = {
            'model': self.model,
            'encoders': self.encoders,
            'feature_columns': self.feature_columns,
            'metrics': self.metrics
        }
        
        with open(filepath, 'wb') as f:
            pickle.dump(model_data, f)
        
        print(f"\nModel saved to {filepath}")
    
    def load_model(self, filepath='poi_cost_model.pkl'):
        """Load a trained model."""
        import pickle
        
        with open(filepath, 'rb') as f:
            model_data = pickle.load(f)
        
        self.model = model_data['model']
        self.encoders = model_data['encoders']
        self.feature_columns = model_data['feature_columns']
        self.metrics = model_data['metrics']
        
        print(f"\nModel loaded from {filepath}")


# Example usage
if __name__ == "__main__":
    
    # Initialize predictor
    predictor = POICostPredictor()
    
    # Load and prepare data
    df = predictor.load_data('interconnectionNY_cost.csv')
    df_clean = predictor.clean_data(df)
    X, y = predictor.prepare_features(df_clean)
    
    # Train model (try different model types: 'random_forest', 'gradient_boosting', 'linear_regression')
    X_train, X_test, y_train, y_test, y_train_pred, y_test_pred = predictor.train_model(
        X, y, model_type='random_forest'
    )
    
    # Plot results
    predictor.plot_results(y_test, y_test_pred, df_clean)
    
    # Make a single prediction
    print("\n" + "="*50)
    print("EXAMPLE PREDICTION")
    print("="*50)
    prediction = predictor.predict_single(
        county='Suffolk',
        resource_type='Solar',
        nameplate_mw=100,
        transmission_owner='Long Island Power Authority',
        study_type='System Impact',
        queue_year=2023
    )
    print(f"\nPredicted POI Cost: ${prediction:.2f}/kW")
    print("="*50)
    
    # Save model
    predictor.save_model('poi_cost_model.pkl')
    
    print("\n✓ Model training complete!")
    print("✓ Use predictor.predict_single() to make new predictions")
    print("✓ Use predictor.save_model() to save the trained model")
    print("✓ Use predictor.load_model() to load a saved model")

Loading data from interconnectionNY_cost.csv...

Dataset shape: (294, 14)
Columns: ['Queue ID', 'Queue ID 2', 'Queue Date', 'State', 'County', 'Resource Type', 'Nameplate MW', 'Interconnecting Transmission Owner', 'Study Type', 'Study Date', 'Request Status', '$2022 POI Cost/kW', '$2022 Network Cost/kW', '$2022 Total Cost/kW']

Cleaning data...


ValueError: could not convert string to float: ' -   '